In [ ]:
import random

def generate_sql_injections(num_samples=1000):
    injections = []
    usernames = ['admin', 'user', 'guest', 'test', 'root', 'alice', 'bob', 'charlie', 'eve', 'mallory']

    for _ in range(num_samples):
        username = random.choice(usernames)

        # List of SQL injection patterns
        injection_patterns = [
            f"SELECT * FROM users WHERE username = '{username}' --",
            f"SELECT * FROM users WHERE username = '{username}' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = '' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT * FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' OR EXISTS(SELECT * FROM users);",
            f"SELECT * FROM users WHERE username = '{username}' OR 1=1 LIMIT 1; --",
            f"SELECT * FROM users WHERE username = '{username}' AND 'a'='a' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' OR (SELECT COUNT(*) FROM users) > 0;",
            f"SELECT * FROM users WHERE username = '{username}' AND (SELECT user FROM users WHERE '1'='1')='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = 'password' OR 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' AND (SELECT 1 FROM dual) IS NOT NULL;",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT username, password FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND '1'='2' UNION SELECT 'hacked', '123';"
        ]

        injection = random.choice(injection_patterns)
        injections.append(injection)

    return injections

# Generate the SQL injection samples
sql_injection_data = generate_sql_injections(1000)

# Save to a file
with open('sql_injection_samples.txt', 'w') as f:
    for line in sql_injection_data:
        f.write(line + '\n')

print("Generated 1000 SQL injection samples and saved to sql_injection_samples.txt.")


: 

In [ ]:
import pandas as pd

# Load the SQL injection samples
with open('sql_injection_samples.txt', 'r') as file:
    injections = [(line.strip(), 1) for line in file.readlines()]  # Mark as harmful

# Load some benign queries as well (for demonstration)
benign_data = [
    ("SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';", 0),
    ("SELECT username, email FROM users WHERE id = 1;", 0),
    ("SELECT COUNT(*) FROM users;", 0),
]

# Combine the datasets
data = injections + benign_data
df = pd.DataFrame(data, columns=['query', 'label'])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[201]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       201

    accuracy                           1.00       201
   macro avg       1.00      1.00      1.00       201
weighted avg       1.00      1.00      1.00       201



In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 100.00%


In [ ]:
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Example usage
print(predict_injection("SELECT * FROM users WHERE username = 'admin' --"))
print(predict_injection("SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';"))


Harmful
Harmful


In [ ]:
# List of harmless SQL queries to test
harmless_queries = [
    "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
    "SELECT username, email FROM users WHERE id = 1;",
    "SELECT COUNT(*) FROM users;",
    "SELECT * FROM products WHERE price < 100;",
    "SELECT name FROM employees WHERE department = 'sales';"
]

# Test the model with harmless queries
for query in harmless_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")


Query: SELECT * FROM users WHERE username = 'alice' AND password = 'securePass'; -> Prediction: Harmful
Query: SELECT username, email FROM users WHERE id = 1; -> Prediction: Harmful
Query: SELECT COUNT(*) FROM users; -> Prediction: Harmful
Query: SELECT * FROM products WHERE price < 100; -> Prediction: Harmful
Query: SELECT name FROM employees WHERE department = 'sales'; -> Prediction: Harmful


The IMmroved version

In [ ]:
import random
import pandas as pd

# Function to generate harmful SQL injection queries
def generate_harmful_queries(num_samples=1000):
    usernames = ['admin', 'user', 'guest', 'test', 'root', 'alice', 'bob', 'charlie', 'eve', 'mallory']
    injections = []

    for _ in range(num_samples):
        username = random.choice(usernames)
        injection_patterns = [
            f"SELECT * FROM users WHERE username = '{username}' --",
            f"SELECT * FROM users WHERE username = '{username}' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = '' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT * FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' OR EXISTS(SELECT * FROM users);",
            f"SELECT * FROM users WHERE username = '{username}' OR 1=1 LIMIT 1; --",
            f"SELECT * FROM users WHERE username = '{username}' AND '1'='2' UNION SELECT 'hacked', '123';"
        ]
        injection = random.choice(injection_patterns)
        injections.append((injection, 1))  # Mark as harmful (1)

    return injections

# Function to generate harmless SQL queries
def generate_harmless_queries(num_samples=1000):
    benign_queries = [
        "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
        "SELECT username, email FROM users WHERE id = 1;",
        "SELECT COUNT(*) FROM users;",
        "SELECT * FROM products WHERE price < 100;",
        "SELECT name FROM employees WHERE department = 'sales';",
        "INSERT INTO users (username, password) VALUES ('testUser', 'testPass');",
        "UPDATE users SET last_login = NOW() WHERE username = 'bob';",
        "DELETE FROM users WHERE id = 10;"
    ]

    harmless_samples = []
    for _ in range(num_samples):
        harmless_query = random.choice(benign_queries)
        harmless_samples.append((harmless_query, 0))  # Mark as harmless (0)

    return harmless_samples

# Generate both harmful and harmless queries
harmful_data = generate_harmful_queries(1000)
harmless_data = generate_harmless_queries(1000)

# Combine data and create DataFrame
data = harmful_data + harmless_data
df = pd.DataFrame(data, columns=['query', 'label'])

# Shuffle the DataFrame to mix harmful and harmless queries
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to a CSV file
df.to_csv('sql_queries_dataset.csv', index=False)

print("Generated 2000 SQL queries (1000 harmful and 1000 harmless) and saved to sql_queries_dataset.csv.")


Generated 2000 SQL queries (1000 harmful and 1000 harmless) and saved to sql_queries_dataset.csv.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load the dataset
df = pd.read_csv('sql_queries_dataset.csv')

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


[[203   0]
 [  0 197]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       203
           1       1.00      1.00      1.00       197

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

Model Accuracy: 100.00%


In [ ]:
# List of harmful SQL queries to test
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';"
]

# List of harmless SQL queries to test
harmless_queries = [
    "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
    "SELECT username, email FROM users WHERE id = 1;",
    "SELECT COUNT(*) FROM users;",
    "SELECT * FROM products WHERE price < 100;",
    "INSERT INTO users (username, password) VALUES ('newUser', 'newPass');"
]

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Test the model with harmful queries
print("Testing Harmful Queries:")
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")

# Test the model with harmless queries
print("\nTesting Harmless Queries:")
for query in harmless_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")


Testing Harmful Queries:
Query: SELECT * FROM users WHERE username = 'admin' -- -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'user' OR '1'='1'; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1'; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'bob' AND '1'='1'; -> Prediction: Harmful

Testing Harmless Queries:
Query: SELECT * FROM users WHERE username = 'alice' AND password = 'securePass'; -> Prediction: Benign
Query: SELECT username, email FROM users WHERE id = 1; -> Prediction: Benign
Query: SELECT COUNT(*) FROM users; -> Prediction: Benign
Query: SELECT * FROM products WHERE price < 100; -> Prediction: Benign
Query: INSERT INTO users (username, password) VALUES ('newUser', 'newPass'); -> Prediction: Benign


Adding llms to report harmful behavoirs

In [ ]:
pip install groq

: 

In [ ]:
from groq import Groq

api_key= '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'
# Function to send an alert email
def send_alert_email(harmful_query):
    client = Groq(api_key = api_key)
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "This is an LLM for reporting SQL injections to Mr. Haggai. Create an email for the alert."
            },
            {
                "role": "user",
                "content": f"A harmful SQL injection has been detected: {harmful_query}"
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    email_content = ""
    for chunk in completion:
        email_content += chunk.choices[0].delta.content or ""

    print("Sending email alert...")
    print(email_content)

# Modify the prediction testing code
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)


ModuleNotFoundError: No module named 'groq'

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_alert_email(harmful_query):
    sender_email = "kondwaninyirenda99@gmail.com"  # Your Gmail address
    receiver_email = "lin3nil0@gmail.com"  # Recipient's email address
    password = "tpqn dtnc prux ffal"  # Your Gmail password or App Password

    # Create the email content
    subject = "SQL Injection Alert"
    body = f"""
    Dear Mr. Haggai,

    A harmful SQL injection has been detected:

    Query: {harmful_query}

    Please take necessary action.

    Best regards,
    Your Security System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        print("Email alert sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")


In [ ]:
# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

In [ ]:
# Define harmful queries
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';"
]

# Test the model with harmful queries
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)

NameError: name 'vectorizer' is not defined

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load the dataset
df = pd.read_csv('sql_queries_dataset.csv')

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Function to send an email alert
def send_alert_email(harmful_query):
    sender_email = "kondwaninyirenda99@gmail.com"  # Your Gmail address
    receiver_email = "lin3nil0@gmail.com"  # Recipient's email address
    password = "tpqn dtnc prux ffal"  # Your Gmail password or App Password

    # Create the email content
    subject = "SQL Injection Alert"
    body = f"""
    Dear Mr. Haggai,

    A harmful SQL injection has been detected:

    Query: {harmful_query}

    Please take necessary action.

    Best regards,
    Your Security System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        print("Email alert sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

# Define harmful queries
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';"
]

# Test the model with harmful queries
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)


FileNotFoundError: [Errno 2] No such file or directory: 'sql_queries_dataset.csv'